In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


/home/tomasz_bednarski/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
mnist = input_data.read_data_sets("../MNIST_data/", one_hot=True)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Print the shapes of images from each subset
print ("IMAGES:")
print (mnist.train.images.shape)
print (mnist.test.images.shape)
print (mnist.validation.images.shape)

# Print the shapes of labels from each subset
print ("LABELS:")
print (mnist.train.labels.shape)
print (mnist.test.labels.shape)
print (mnist.validation.labels.shape)

IMAGES:
(55000, 784)
(10000, 784)
(5000, 784)
LABELS:
(55000, 10)
(10000, 10)
(5000, 10)


In [8]:
mnist.train.images[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [9]:
mnist.train.labels[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])

In [4]:
x = tf.placeholder(tf.float32, [None, 784])   # None - dim can be any length

A Variable is a modifiable tensor that lives in TensorFlow's graph of interacting operations. It can be used and even modified by the computation. For machine learning applications, one generally has the model parameters be Variables.

In [5]:
# initialized with 0s
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))


Model

In [6]:
# matrix multiplication x*W [many,784] * [784,10] = [many, 10]
# and softmax
y = tf.nn.softmax(tf.matmul(x, W) + b)


Reduction func

In [17]:
sess1 = tf.Session()
matr = tf.constant([[1., 1.], [2., 2.]])
print(sess1.run(matr))
tf.reduce_mean(matr)  # 1.5
tf.reduce_mean(matr, 0)  # [1.5, 1.5]
tf.reduce_mean(matr, 1)  # [1.,  2.]

[[ 1.  1.]
 [ 2.  2.]]


Training

In [7]:
# cost function - cross entropy
y_true = tf.placeholder(tf.float32, [None, 10])

#can be numerically unstable
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(y), reduction_indices=[1]))

# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y))


# minimize loss with learning rate 0.5
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)


In [8]:
sess = tf.InteractiveSession()

In [9]:
tf.global_variables_initializer().run()

In [10]:
for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_true: batch_ys})

Evaluation

Well, first let's figure out where we predicted the correct label. tf.argmax is an extremely useful function which gives you the index of the highest entry in a tensor along some axis. For example, tf.argmax(y,1) is the label our model thinks is most likely for each input, while tf.argmax(y_true,1) is the correct label. We can use tf.equal to check if our prediction matches the truth.

In [12]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_true,1))

In [13]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_true: mnist.test.labels}))

0.9062
